In [ ]:
import plots

# Fluxes

In [ ]:
f = plots.benchmark('fluxes', '60lev', ymin=0.4, server='uriah')

In [ ]:
f = plots.benchmark('fluxes', '60lev', ncols=2, ymin=0.4, ymax=100, server='cheyenne4')

# Dimension slicing

In [ ]:
f = plots.benchmark('slice', '60lev', ncols=4, ymin=0.05, ymax=8, server='uriah')

# Interpolation

In [ ]:
f = plots.benchmark('isentropes', '60lev', ncols=4, ymin=0.8, ymax=150, server='uriah')